# CLAPES UC

## Extracción de links

Seminarios

In [6]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def extract_articles_from_url2(driver, url, existing_df=None):
    """
    Extrae información de artículos desde una URL y agrega los resultados a un DataFrame existente.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): URL de la página para extraer artículos.
        existing_df (pd.DataFrame, optional): DataFrame al que agregar los artículos extraídos. Si no se proporciona, se crea uno nuevo.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos agregados al existente (o uno nuevo si no se proporciona).
    """
    try:
        driver.get(url)
        # Esperar a que cargue el contenido
        time.sleep(3)

        # Obtener todos los artículos dentro de los divs con clase 'col-sm-6 mb-32'
        articles = driver.find_elements(By.CSS_SELECTOR, "div.col-sm-6.mb-32")

        data = []
        for article in articles:
            # Imagen
            try:
                img = article.find_element(By.CSS_SELECTOR, "span img")
                image_url = img.get_attribute("src")
                # Asegurarse de que la URL de la imagen sea completa
                if image_url and not image_url.startswith("http"):
                    image_url = "https://example.com" + image_url  # Reemplaza con el dominio adecuado
            except:
                image_url = None

            # Título y enlace
            try:
                title_element = article.find_element(By.CSS_SELECTOR, "h4.h3 a")
                title = title_element.text.strip()
                link = title_element.get_attribute("href")
            except:
                continue  # Si no hay título o enlace, saltamos este artículo

            # Fecha
            try:
                date = article.find_element(By.CSS_SELECTOR, "p.p-size--sm span").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'URL Imagen': image_url,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}?page={current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

In [49]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scroll_to_bottom(driver):
    """Desplaza la página hasta el final y espera un segundo para cargar el contenido dinámico."""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Esperar un segundo para que la página cargue

def extract_articles_from_url2(driver, url, existing_df=None):
    try:
        driver.get(url)

        # Desplazar hasta el final para cargar contenido dinámico
        scroll_to_bottom(driver)
        time.sleep(1)
        scroll_to_bottom(driver)
        time.sleep(1)

        # Obtener todos los divs con clase 'col-sm-6'
        columns = driver.find_elements(By.CSS_SELECTOR, "div.col-sm-6")

        data = []
        for column in columns:
            # Buscar el enlace dentro del div
            try:
                # Extraer el enlace del artículo dentro del <a> que está después de <span>
                link_element = column.find_element(By.CSS_SELECTOR, "a.uc-card_link")
                link = link_element.get_attribute("href")
                # Si el enlace es relativo, agregar el dominio base
                if link.startswith('/'):
                    link = f"https://www.clapesuc.cl{link}"
            except:
                link = "No disponible"  # Si no hay enlace, asignamos un valor predeterminado

            # Obtener el título
            try:
                title_element = column.find_element(By.CSS_SELECTOR, "h4.h3")
                title = title_element.text.strip()
            except:
                title = "No disponible"

            # Obtener el autor
            try:
                author = column.find_element(By.CSS_SELECTOR, "p.text-weight--bold.no-margin").text.strip()
            except:
                author = "No especificado"

            # Obtener la fecha
            try:
                date = column.find_element(By.CSS_SELECTOR, "p.p-size--sm span").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            article_data = {
                'Título': title,
                'Enlace': link,
                'Autor': author,
                'Fecha Publicación': date,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(article_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los artículos, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los artículos extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_articles_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}?page={current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo artículos de: {url}")

        # Llamar a la función extract_articles_from_url2 para obtener artículos de la página
        all_articles_df = extract_articles_from_url2(driver, url, all_articles_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_articles_df is None or len(all_articles_df) == 0:
            print(f"No se encontraron artículos en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_articles_df

Seminarios

In [50]:
base_url = "https://clapesuc.cl/seminarios"
seminarios = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=30, headless=False)

Extrayendo artículos de: https://clapesuc.cl/seminarios?page=1
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=2
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=3
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=4
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=5
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=6
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=7
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=8
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=9
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=10
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=11
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=12
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=13
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=14
Extrayendo artículos de: https://clapesuc.cl/seminarios?page=15
Extrayendo artículos de: https://clapesuc.cl/semi

In [53]:
seminarios

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,None,2024-11-29 13:57:31
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,None,2024-11-29 13:57:32
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,None,2024-11-29 13:57:32
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,None,2024-11-29 13:57:32
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,None,2024-11-29 13:57:32
...,...,...,...,...,...,...
254,[Seminario Reforma Tributaria] PPT Principios ...,https://clapesuc.cl/seminarios/seminario-refor...,Claudio Agostini,13 de mayo del 2014,None,2024-11-29 14:00:11
255,[Seminario Educación] PPT Sobre los mitos de ...,https://clapesuc.cl/seminarios/seminario-educa...,Claudia Sanhueza,13 de mayo del 2014,None,2024-11-29 14:00:11
256,[Seminario Educación] PPT Reflexiones sobre l...,https://clapesuc.cl/seminarios/seminario-educa...,Ricardo Paredes,13 de mayo del 2014,None,2024-11-29 14:00:11
257,[Seminario Educación] PPT Gratuidad en Educac...,https://clapesuc.cl/seminarios/seminario-educa...,Sergio Urzúa,13 de mayo del 2014,None,2024-11-29 14:00:11


In [54]:
seminarios.to_excel("seminarios_uc.xlsx", index=False)

Investigación

In [51]:
base_url = "https://clapesuc.cl/investigacion"
investigacion = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=106, headless=False)
investigacion

Extrayendo artículos de: https://clapesuc.cl/investigacion?page=1
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=2
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=3
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=4
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=5
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=6
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=7
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=8
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=9
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=10
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=11
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=12
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=13
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=14
Extrayendo artículos de: https://clapesuc.cl/investigacion?page=15
Extr

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta
0,"Ideas para el Debate 38: ""Perspectivas para la...",https://clapesuc.cl/investigacion/ideas-para-e...,No especificado,26 de noviembre del 2024,None,2024-11-29 14:00:25
1,Informe: Impacto del Teletrabajo en Productivi...,https://clapesuc.cl/investigacion/informe-impa...,No especificado,18 de noviembre del 2024,None,2024-11-29 14:00:25
2,Informe de empleo formal en el sector privado ...,https://clapesuc.cl/investigacion/informe-de-e...,No especificado,14 de noviembre del 2024,None,2024-11-29 14:00:25
3,"Ideas para el Debate 37: ""Presentaciones del L...",https://clapesuc.cl/investigacion/ideas-para-e...,No especificado,12 de noviembre del 2024,None,2024-11-29 14:00:25
4,Estimación del Valor de la CBA,https://clapesuc.cl/investigacion/estimacion-d...,No especificado,12 de noviembre del 2024,None,2024-11-29 14:00:25
...,...,...,...,...,...,...
943,"Doc.Trabajo Nº2, La utilidad atribuida: una cr...",https://clapesuc.cl/investigacion/doc-trabajo-...,No especificado,3 de junio del 2014,None,2024-11-29 14:09:55
944,No disponible,No disponible,No especificado,No especificada,None,2024-11-29 14:09:55
945,Análisis Impuesto al Capital durante la Gran D...,https://clapesuc.cl/investigacion/analisis-imp...,No especificado,29 de mayo del 2014,None,2024-11-29 14:09:59
946,"Doc.Trabajo Nº1, Reforma Tributaria: Impacto E...",https://clapesuc.cl/investigacion/doc-trabajo-...,No especificado,15 de mayo del 2014,None,2024-11-29 14:10:00


In [55]:
investigacion.to_excel("investigacion_uc.xlsx", index=False)

En los medios

In [ ]:
base_url = "https://clapesuc.cl/en-los-medios"
elm = extract_articles_from_multiple_pages(base_url, start_page=1, max_pages=456, headless=False)
elm

Extrayendo artículos de: https://clapesuc.cl/en-los-medios?page=1


Indicadores

In [47]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime
from selenium.webdriver.chrome.options import Options

def create_driver(headless=True):
    """Crea y retorna un WebDriver con la opción de ser headless si se desea."""
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    driver = webdriver.Chrome(options=chrome_options)
    return driver

def scroll_to_bottom(driver):
    """Desplaza la página hasta el final y espera un segundo para cargar el contenido dinámico."""
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)  # Esperar un segundo para que la página cargue

def extract_indicators_from_url(driver, url, existing_df=None):
    try:
        driver.get(url)

        # Desplazar hasta el final para cargar contenido dinámico
        scroll_to_bottom(driver)
        time.sleep(1)
        scroll_to_bottom(driver)
        time.sleep(1)

        # Obtener todos los divs con clase 'uc-card card-type--link'
        cards = driver.find_elements(By.CSS_SELECTOR, "div.uc-card.card-type--link")

        data = []
        for card in cards:
            # Extraer el enlace del indicador
            try:
                link_element = card.find_element(By.CSS_SELECTOR, "a.uc-card_link")
                link = link_element.get_attribute("href")
                # Si el enlace es relativo, agregar el dominio base
                if link.startswith('/'):
                    link = f"https://www.clapesuc.cl{link}"
            except:
                link = "No disponible"  # Si no hay enlace, asignamos un valor predeterminado

            # Obtener el título del indicador
            try:
                title_element = card.find_element(By.CSS_SELECTOR, "h4.h3")
                title = title_element.text.strip()
            except:
                title = "No disponible"

            # Obtener la fecha de publicación
            try:
                date = card.find_element(By.CSS_SELECTOR, "p.p-size--sm.p-color--gray.p-text--condensed span").text.strip()
            except:
                date = "No especificada"

            # Crear diccionario con los datos
            indicator_data = {
                'Título': title,
                'Enlace': link,
                'Fecha Publicación': date,
                'Contenido': None,  # El contenido no parece estar disponible en esta estructura
                'Fecha Consulta': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            }

            data.append(indicator_data)

        # Crear DataFrame con los nuevos datos
        new_df = pd.DataFrame(data)

        # Si ya existe un DataFrame proporcionado, concatenar los nuevos datos al existente
        if existing_df is not None:
            existing_df = pd.concat([existing_df, new_df], ignore_index=True)
        else:
            # Si no existe, devolver el nuevo DataFrame
            existing_df = new_df

        return existing_df

    except Exception as e:
        print(f"Error en la extracción: {str(e)}")
        return existing_df if existing_df is not None else pd.DataFrame()

def extract_indicators_from_multiple_pages(base_url, start_page=1, max_pages=100, headless=True):
    """
    Itera sobre varias páginas y extrae los indicadores, deteniéndose cuando una página no se encuentra.
    
    Args:
        base_url (str): URL base de las páginas, debe contener un marcador de página, e.g., 'page/{page_number}/'.
        start_page (int): Número de página inicial para empezar a iterar (por defecto es la página 1).
        max_pages (int): Número máximo de páginas a intentar antes de detener la búsqueda (por defecto es 100).
        headless (bool): Si se debe ejecutar el navegador en modo headless (sin abrir ventana). Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con los indicadores extraídos de todas las páginas procesadas.
    """
    driver = create_driver(headless)  # Inicializar el WebDriver solo una vez
    current_page = start_page
    all_indicators_df = pd.DataFrame()  # Inicializamos un DataFrame vacío

    while current_page <= max_pages:
        # Construir la URL de la página actual
        url = f"{base_url}?page={current_page}"  # Aquí el formato de la URL, el número de página se añade al final
        print(f"Extrayendo indicadores de: {url}")

        # Llamar a la función extract_indicators_from_url para obtener indicadores de la página
        all_indicators_df = extract_indicators_from_url(driver, url, all_indicators_df)

        # Verificar si se encontró una página de error (por ejemplo, página no encontrada)
        if all_indicators_df is None or len(all_indicators_df) == 0:
            print(f"No se encontraron indicadores en la página {current_page}, o la página no existe.")
            break

        current_page += 1  # Pasar a la siguiente página

    driver.quit()  # Cerrar el WebDriver después de todas las iteraciones
    return all_indicators_df

In [48]:
base_url = "https://clapesuc.cl/indicadores"
indicadores = extract_indicators_from_multiple_pages(base_url, start_page=1, max_pages=1, headless=False)
indicadores

Extrayendo indicadores de: https://clapesuc.cl/indicadores?page=1


,Título,Enlace,Fecha Publicación,Contenido,Fecha Consulta
0,Índice de Precios de Medicamentos (IPC Med) CL...,https://clapesuc.cl/indicadores/indice-de-prec...,14 de noviembre del 2024,None,2024-11-29 13:56:27
1,Incertidumbre Económica (IEC) y Política Econó...,https://clapesuc.cl/indicadores/indice-de-ince...,14 de noviembre del 2024,None,2024-11-29 13:56:27
2,Probabilidad de Recesión en Chile,https://clapesuc.cl/indicadores/probabilidad-d...,14 de noviembre del 2024,None,2024-11-29 13:56:27
3,Índice de Condiciones del Mercado Laboral (ICML),https://clapesuc.cl/indicadores/indice-de-cond...,14 de noviembre del 2024,None,2024-11-29 13:56:27
4,IPC del Adulto Mayor,https://clapesuc.cl/indicadores/ipc-adulto-mayor,12 de noviembre del 2024,None,2024-11-29 13:56:27
5,Índice de Precios al Consumidor de Escasos Ing...,https://clapesuc.cl/indicadores/indice-de-prec...,8 de noviembre del 2024,None,2024-11-29 13:56:27
6,Precios Inmobiliario Residencial,https://clapesuc.cl/indicadores/indice-inmobil...,9 de octubre del 2024,None,2024-11-29 13:56:27
7,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,21 de septiembre del 2024,None,2024-11-29 13:56:27
8,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,17 de septiembre del 2024,None,2024-11-29 13:56:27
9,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,13 de septiembre del 2024,None,2024-11-29 13:56:27


In [56]:
indicadores.to_excel("indicadores_uc.xlsx", index=False)

In [59]:
import pandas as pd

# Lista de DataFrames
dataframes = [seminarios, indicadores]

# Añadir una columna que indique el origen de cada DataFrame
seminarios['Categoría'] = 'Seminarios'
indicadores['Categoría'] = 'Indicadores'

# Concatenar los DataFrames
df_concatenado0 = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado0

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta,Categoría
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,None,2024-11-29 13:57:31,Seminarios
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,None,2024-11-29 13:57:32,Seminarios
...,...,...,...,...,...,...,...
266,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,NaN,21 de septiembre del 2024,None,2024-11-29 13:56:27,Indicadores
267,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,NaN,17 de septiembre del 2024,None,2024-11-29 13:56:27,Indicadores
268,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,NaN,13 de septiembre del 2024,None,2024-11-29 13:56:27,Indicadores
269,Índice de Acceso a la Vivienda (IAV),https://clapesuc.cl/indicadores/indice-de-acce...,NaN,30 de agosto del 2024,None,2024-11-29 13:56:27,Indicadores


In [60]:
df_concatenado0.to_excel("sem_ind_uc.xlsx", index=False)

## Consolidación misma estructura

Agregar investigación al final porque no tiene informacion dentro del link, solo un pdf.

In [ ]:
import pandas as pd

# Lista de DataFrames
dataframes = [seminarios, elm, indicadores]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,URL Imagen,Fecha,Fecha Consulta
0,ALERTA TEMPRANA N°46: NUEVO PARO DE FUNCIONARI...,https://www.respublica.cl/articulo/1155,https://www.respublica.cl/img/uploads/36183741...,"28 de Noviembre, 2024",2024-11-29 10:09:46
1,ALERTA TEMPRANA N°45 NUEVO ESCENARIO POLÍTICO ...,https://www.respublica.cl/articulo/1151,https://www.respublica.cl/img/uploads/0d6e962e...,"25 de Noviembre, 2024",2024-11-29 10:09:46
2,ALERTA TEMPRANA N°44 USO DE RECURSOS FISCALES ...,https://www.respublica.cl/articulo/1149,https://www.respublica.cl/img/uploads/3ec61926...,"21 de Noviembre, 2024",2024-11-29 10:09:46
3,ALERTA TEMPRANA N°43 LAS INCOHERENCIAS DEL GOB...,https://www.respublica.cl/articulo/1150,https://www.respublica.cl/img/uploads/8f6b03c8...,"21 de Noviembre, 2024",2024-11-29 10:09:47
4,ALERTA TEMPRANA N°42 ¿ES LEGAL UN PARO DE FUNC...,https://www.respublica.cl/articulo/1146,https://www.respublica.cl/img/uploads/f510a91f...,"19 de Noviembre, 2024",2024-11-29 10:09:47
...,...,...,...,...,...
670,Pensiones en el Blanco,https://www.respublica.cl/articulo-analisis/220,https://www.respublica.cl/img/uploads/a9e54846...,"01 de Agosto, 2020",2024-11-29 10:14:34
671,Migración en Chile: Necesidad de una nueva ley,https://www.respublica.cl/articulo-analisis/225,https://www.respublica.cl/img/uploads/a3d41ec5...,"28 de Agosto, 2020",2024-11-29 10:14:34
672,Reflexiones sobre el plebiscito,https://www.respublica.cl/articulo-analisis/241,https://www.respublica.cl/img/uploads/f8de70fb...,"17 de Septiembre, 2020",2024-11-29 10:14:34
673,Letalidad del COVID-19 y Pobreza Multidimensio...,https://www.respublica.cl/articulo-analisis/242,https://www.respublica.cl/img/uploads/c82b8dfd...,"07 de Octubre, 2020",2024-11-29 10:14:34


## Iteración sobre links

In [2]:
import pandas as pd

df_concatenado = pd.read_excel("sem_ind_uc.xlsx")
df_concatenado

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta,Categoría
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,NaN,2024-11-29 13:57:31,Seminarios
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
...,...,...,...,...,...,...,...
266,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,NaN,21 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
267,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,NaN,17 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
268,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,NaN,13 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
269,Índice de Acceso a la Vivienda (IAV),https://clapesuc.cl/indicadores/indice-de-acce...,NaN,30 de agosto del 2024,NaN,2024-11-29 13:56:27,Indicadores


In [6]:
from selenium.webdriver.common.by import By

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (ajustar según la velocidad de carga)
    driver.implicitly_wait(1)
    
    # Intentar obtener la información:
    
    # Título del seminario (primer párrafo con texto centrado y fuerte)
    try:
        title = driver.find_element(By.CSS_SELECTOR, "div p[style='text-align: center;'] strong").text.strip()
    except:
        title = "No disponible"
    
    # Fecha de publicación (párrafo con texto alineado a la derecha)
    try:
        publication_date = driver.find_element(By.CSS_SELECTOR, "div p[style='text-align: right;'] strong").text.strip()
    except:
        publication_date = "No disponible"
    
    # Contenido del seminario (párrafos con texto justificado)
    try:
        paragraphs = driver.find_elements(By.CSS_SELECTOR, "div p[style='text-align: justify;']")
        content = " ".join([p.text.strip() for p in paragraphs])
    except:
        content = "No disponible"
    
    # Participantes (en el div con 'src="true"')
    try:
        participants = driver.find_element(By.CSS_SELECTOR, "div[src='true'] span").text.strip()
    except:
        participants = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Fecha de Publicación": publication_date,
        "Contenido": content,
        "Participantes": participants,
        "Enlace": url,
    }
    
    return article_data

In [9]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm  # Importar tqdm para la barra de carga

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.

    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Si es True, se ejecutará en modo headless (sin interfaz gráfica). Si es False, con interfaz gráfica.

    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless si 'headless' es True
    chrome_options = Options()
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        # Usar tqdm para mostrar la barra de carga mientras se procesan los enlaces
        for _, row in tqdm(jg_df.iterrows(), total=jg_df.shape[0], desc="Extrayendo datos de artículos"):
            url = row['Enlace']
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [7]:
aver = df_concatenado[:3]
averinfo = process_all_links(aver)
averinfo

Extrayendo datos de artículos: 100%|██████████| 3/3 [00:05<00:00,  1.94s/it]


,Título,Fecha de Publicación,Contenido,Participantes,Enlace
0,Seminario CLAPES UC “Estrategia Nacional del L...,"05 de noviembre, 2024",La estrategia nacional del litio fue lanzada p...,"José Miguel Benavente, Hernán de Solminihac, D...",https://clapesuc.cl/seminarios/seminario-clape...
1,Seminario CLAPES UC “Perspectivas para las Fin...,"29 de octubre, 2024",Uno de los principales atributos institucional...,"Cecilia Cifuentes, Hermann González, Felipe La...",https://clapesuc.cl/seminarios/seminario-finan...
2,"Presentación del Libro: ""Chile 2050: Un País. ...","17 de octubre, 2024",La actividad se realizó el jueves 17 de octubr...,"Andrés Allamand, Cayetana Álvarez de Toledo,",https://clapesuc.cl/seminarios/presentacion-de...


Para todos pero dara error para varias filas por fecha y programa, correr de nuevo con esa estructura de error.

In [11]:
# Filtrar las filas donde la columna 'Enlace' no sea igual a 'No disponible'
df_concatenado = df_concatenado[df_concatenado['Enlace'] != 'No disponible']
df_concatenado

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta,Categoría
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,NaN,2024-11-29 13:57:31,Seminarios
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
...,...,...,...,...,...,...,...
266,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,NaN,21 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
267,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,NaN,17 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
268,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,NaN,13 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
269,Índice de Acceso a la Vivienda (IAV),https://clapesuc.cl/indicadores/indice-de-acce...,NaN,30 de agosto del 2024,NaN,2024-11-29 13:56:27,Indicadores


In [12]:
df_info = process_all_links(df_concatenado, headless=False)
df_info

Extrayendo datos de artículos: 100%|██████████| 241/241 [20:11<00:00,  5.03s/it]


,Título,Fecha de Publicación,Contenido,Participantes,Enlace
0,Seminario CLAPES UC “Estrategia Nacional del L...,"05 de noviembre, 2024",La estrategia nacional del litio fue lanzada p...,"José Miguel Benavente, Hernán de Solminihac, D...",https://clapesuc.cl/seminarios/seminario-clape...
1,Seminario CLAPES UC “Perspectivas para las Fin...,"29 de octubre, 2024",Uno de los principales atributos institucional...,"Cecilia Cifuentes, Hermann González, Felipe La...",https://clapesuc.cl/seminarios/seminario-finan...
2,"Presentación del Libro: ""Chile 2050: Un País. ...","17 de octubre, 2024",La actividad se realizó el jueves 17 de octubr...,"Andrés Allamand, Cayetana Álvarez de Toledo,",https://clapesuc.cl/seminarios/presentacion-de...
3,"Evento ""Reforzando el Futuro de la Democracia ...","16 de octubre, 2024",La actividad organizada en conjunto con el Cen...,"Diego Area, Ana Castellanos, Jason Marczak, , ...",https://clapesuc.cl/seminarios/evento-reforzan...
4,Seminario “Financiamiento de la Educación Supe...,"15 de octubre, 2024",La actividad organizada en conjunto con el Cen...,"Carlos Williamson, Harald Beyer, Claudia Peira...",https://clapesuc.cl/seminarios/seminario-educa...
...,...,...,...,...,...
236,No disponible,No disponible,Este Indicador de Productividad CLAPES UC se ...,No disponible,https://clapesuc.cl/indicadores/indice-de-prod...
237,No disponible,No disponible,El Índice de Miseria es un indicador simple qu...,No disponible,https://clapesuc.cl/indicadores/indice-de-mise...
238,No disponible,No disponible,Desde CLAPES UC se pone a disposición del públ...,No disponible,https://clapesuc.cl/indicadores/indicador-de-f...
239,No disponible,No disponible,El Índice compuesto de Asequibilidad de la Viv...,No disponible,https://clapesuc.cl/indicadores/indice-de-acce...


In [13]:
df_concatenado

,Título,Enlace,Autor,Fecha Publicación,Contenido,Fecha Consulta,Categoría
0,"Seminario Clapes UC: ""Estrategia Nacional del ...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,NaN,2024-11-29 13:57:31,Seminarios
1,"Seminario Clapes UC: ""Perspectivas para las Fi...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
2,"Presentación del Libro: ""Chile 2050: Un País. ...",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
3,"Evento ""Reforzando el Futuro de la Democracia ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
4,"Seminario ""Financiamiento de la Educación Supe...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,NaN,2024-11-29 13:57:32,Seminarios
...,...,...,...,...,...,...,...
266,Índice de Productividad CLAPES UC,https://clapesuc.cl/indicadores/indice-de-prod...,NaN,21 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
267,Índice de Miseria (IM) para América Latina,https://clapesuc.cl/indicadores/indice-de-mise...,NaN,17 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
268,Indicador de Factores Externos e Internos del ...,https://clapesuc.cl/indicadores/indicador-de-f...,NaN,13 de septiembre del 2024,NaN,2024-11-29 13:56:27,Indicadores
269,Índice de Acceso a la Vivienda (IAV),https://clapesuc.cl/indicadores/indice-de-acce...,NaN,30 de agosto del 2024,NaN,2024-11-29 13:56:27,Indicadores


In [14]:
df_info

,Título,Fecha de Publicación,Contenido,Participantes,Enlace
0,Seminario CLAPES UC “Estrategia Nacional del L...,"05 de noviembre, 2024",La estrategia nacional del litio fue lanzada p...,"José Miguel Benavente, Hernán de Solminihac, D...",https://clapesuc.cl/seminarios/seminario-clape...
1,Seminario CLAPES UC “Perspectivas para las Fin...,"29 de octubre, 2024",Uno de los principales atributos institucional...,"Cecilia Cifuentes, Hermann González, Felipe La...",https://clapesuc.cl/seminarios/seminario-finan...
2,"Presentación del Libro: ""Chile 2050: Un País. ...","17 de octubre, 2024",La actividad se realizó el jueves 17 de octubr...,"Andrés Allamand, Cayetana Álvarez de Toledo,",https://clapesuc.cl/seminarios/presentacion-de...
3,"Evento ""Reforzando el Futuro de la Democracia ...","16 de octubre, 2024",La actividad organizada en conjunto con el Cen...,"Diego Area, Ana Castellanos, Jason Marczak, , ...",https://clapesuc.cl/seminarios/evento-reforzan...
4,Seminario “Financiamiento de la Educación Supe...,"15 de octubre, 2024",La actividad organizada en conjunto con el Cen...,"Carlos Williamson, Harald Beyer, Claudia Peira...",https://clapesuc.cl/seminarios/seminario-educa...
...,...,...,...,...,...
236,No disponible,No disponible,Este Indicador de Productividad CLAPES UC se ...,No disponible,https://clapesuc.cl/indicadores/indice-de-prod...
237,No disponible,No disponible,El Índice de Miseria es un indicador simple qu...,No disponible,https://clapesuc.cl/indicadores/indice-de-mise...
238,No disponible,No disponible,Desde CLAPES UC se pone a disposición del públ...,No disponible,https://clapesuc.cl/indicadores/indicador-de-f...
239,No disponible,No disponible,El Índice compuesto de Asequibilidad de la Viv...,No disponible,https://clapesuc.cl/indicadores/indice-de-acce...


In [15]:
# Identificar las columnas que están en df_concatenado pero no en df_info
columnas_extra = [col for col in df_concatenado.columns if col not in df_info.columns]

# Seleccionar solo las columnas extra de df_concatenado
df_concatenado_extra = df_concatenado[['Enlace'] + columnas_extra]

# Hacer el merge entre df_info y df_concatenado_extra en la columna "Enlace"
df_combinado = pd.merge(df_info, df_concatenado_extra, on='Enlace', how='left')

# Ver el DataFrame combinado
df_combinado

,Título,Fecha de Publicación,Contenido,Participantes,Enlace,Autor,Fecha Publicación,Fecha Consulta,Categoría
0,Seminario CLAPES UC “Estrategia Nacional del L...,"05 de noviembre, 2024",La estrategia nacional del litio fue lanzada p...,"José Miguel Benavente, Hernán de Solminihac, D...",https://clapesuc.cl/seminarios/seminario-clape...,Felipe Larraín,5 de noviembre del 2024,2024-11-29 13:57:31,Seminarios
1,Seminario CLAPES UC “Perspectivas para las Fin...,"29 de octubre, 2024",Uno de los principales atributos institucional...,"Cecilia Cifuentes, Hermann González, Felipe La...",https://clapesuc.cl/seminarios/seminario-finan...,Felipe Larraín,29 de octubre del 2024,2024-11-29 13:57:32,Seminarios
2,"Presentación del Libro: ""Chile 2050: Un País. ...","17 de octubre, 2024",La actividad se realizó el jueves 17 de octubr...,"Andrés Allamand, Cayetana Álvarez de Toledo,",https://clapesuc.cl/seminarios/presentacion-de...,Felipe Larraín,17 de octubre del 2024,2024-11-29 13:57:32,Seminarios
3,"Evento ""Reforzando el Futuro de la Democracia ...","16 de octubre, 2024",La actividad organizada en conjunto con el Cen...,"Diego Area, Ana Castellanos, Jason Marczak, , ...",https://clapesuc.cl/seminarios/evento-reforzan...,Clapes UC,16 de octubre del 2024,2024-11-29 13:57:32,Seminarios
4,Seminario “Financiamiento de la Educación Supe...,"15 de octubre, 2024",La actividad organizada en conjunto con el Cen...,"Carlos Williamson, Harald Beyer, Claudia Peira...",https://clapesuc.cl/seminarios/seminario-educa...,Harald Beyer,15 de octubre del 2024,2024-11-29 13:57:32,Seminarios
...,...,...,...,...,...,...,...,...,...
236,No disponible,No disponible,Este Indicador de Productividad CLAPES UC se ...,No disponible,https://clapesuc.cl/indicadores/indice-de-prod...,NaN,21 de septiembre del 2024,2024-11-29 13:56:27,Indicadores
237,No disponible,No disponible,El Índice de Miseria es un indicador simple qu...,No disponible,https://clapesuc.cl/indicadores/indice-de-mise...,NaN,17 de septiembre del 2024,2024-11-29 13:56:27,Indicadores
238,No disponible,No disponible,Desde CLAPES UC se pone a disposición del públ...,No disponible,https://clapesuc.cl/indicadores/indicador-de-f...,NaN,13 de septiembre del 2024,2024-11-29 13:56:27,Indicadores
239,No disponible,No disponible,El Índice compuesto de Asequibilidad de la Viv...,No disponible,https://clapesuc.cl/indicadores/indice-de-acce...,NaN,30 de agosto del 2024,2024-11-29 13:56:27,Indicadores


In [16]:
df_combinado.columns

Index(['Título', 'Fecha de Publicación', 'Contenido', 'Participantes',
       'Enlace', 'Autor', 'Fecha Publicación', 'Fecha Consulta', 'Categoría'],
      dtype='object')

In [17]:
df_final = df_combinado.copy()
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "CLAPES UC"

df_final  = df_final[['Título', 'Fecha de Publicación', 'Contenido', 'Participantes',
       'Enlace', 'Autor', 'Fecha Publicación', 'Categoría']]

df_final.to_excel("CLAPES UC - 1.xlsx", index = False)

Segunda pasada con nueva función

## Resto de dfs

# Consolidación

# Anterior

In [13]:
ip_elm2 = ip_elm[:2]
ip_lyr2 = ip_lyr[:2]
ip_notins2 = ip_notins[:2]

In [14]:
ip_elm_contenido = process_all_links(ip_elm)
ip_elm_contenido

Extrayendo datos de: https://ideapais.cl/noticias/columna-a-donde-estan-que-no-se-ven-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/carta-al-director-el-parche-de-la-confianza-por-arnau-sarra-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-costos-de-mantener-a-toha-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-volver-a-poner-a-las-personas-en-el-centro-por-matias-domeyko-diario-el-llanquihue/
Extrayendo datos de: https://ideapais.cl/noticias/columna-solidaridad-uc-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-los-catolicos-y-la-conciencia-por-cristian-stewart-la-segunda/
Extrayendo datos de: https://ideapais.cl/noticias/columna-cae-victimas-y-opresores-por-magdalena-vergara-diario-la-tercera/
Extrayendo datos de: https://ideapais.cl/noticias/columna-es-justo-por-juan-pablo-lira-el-dinamo/
Extrayendo datos de: https://ideapais.cl/notici

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,"Columna «A dónde están, que no se ven», por Cr...",IdeaPais,"Nov 21, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Dentro de las varias aristas que tiene el caso...,https://ideapais.cl/noticias/columna-a-donde-e...
1,"Carta al Director «El parche de la confianza»,...",María José Aravena,"Nov 14, 2024","María José Aravena, Noticias, Noticias Destaca...","Señor Director:\nEn su columna, Miriam Henríqu...",https://ideapais.cl/noticias/carta-al-director...
2,"Columna «Los costos de mantener a Tohá», por C...",IdeaPais,"Nov 7, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",A nadie le gusta que le digan cómo uno debe ha...,https://ideapais.cl/noticias/columna-los-costo...
3,Columna «Volver a poner a las personas en el c...,IdeaPais,"Oct 25, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace algunos días, en IdeaPaís organizamos un ...",https://ideapais.cl/noticias/columna-volver-a-...
4,"Columna «Solidaridad UC», por Cristián Stewart...",IdeaPais,"Oct 24, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...","Hace 14 años, nació un proyecto político en la...",https://ideapais.cl/noticias/columna-solidarid...
5,"Columna «Los católicos y la conciencia», por C...",IdeaPais,"Sep 26, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",El profesor Carlos Peña inició una importante ...,https://ideapais.cl/noticias/columna-los-catol...
6,"Columna «CAE: víctimas y opresores», por Magda...",IdeaPais,"Sep 18, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",La manera en que el Frente Amplio ha enarbolad...,https://ideapais.cl/noticias/columna-cae-victi...
7,"Columna «¿Es justo?», por Juan Pablo Lira | El...",IdeaPais,"Sep 17, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Hablemos de la asignación de recursos en educa...,https://ideapais.cl/noticias/columna-es-justo-...
8,"Columna «Conmemoración de hechos importantes»,...",IdeaPais,"Sep 16, 2024","IdeaPais, Noticias, Noticias Destacadas Inicio...",Sin duda que las primeras semanas de septiembr...,https://ideapais.cl/noticias/columna-conmemora...


In [15]:
ip_notins_contenido = process_all_links(ip_notins)
ip_notins_contenido

Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-organiza-conversatorio-sobre-desarrollo-urbano-con-alcaldes-de-puerto-montt-y-puerto-varas/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/candidatos-a-gobernador-regional-de-la-region-de-los-lagos-debatieron-en-el-evento-organizado-por-ideapais/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-invita-a-postular-al-ii-congreso-de-pensamiento-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/ideapais-inaugura-sede-regional-en-los-lagos-es-la-cuarta-a-nivel-nacional/
Extrayendo datos de: https://ideapais.cl/sin-categoria/ideapais-realiza-debates-entre-candidatos-a-alcaldes-en-santa-cruz-y-marchigue/
Extrayendo datos de: https://ideapais.cl/noticias-institucionales/con-presencia-de-evelyn-matthei-el-centro-de-estudios-ideapais-realizo-el-congreso-de-politicas-publicas-profamilia/
Extrayendo datos de: https://ideapais.cl/noti

,Título,Autor,Fecha de Publicación,Categorías,Contenido,Enlace
0,IdeaPaís organiza conversatorio sobre desarrol...,María José Aravena,"Nov 28, 2024","María José Aravena, Noticias Destacadas Inicio...",Rodrigo Wainraihgt y Tomás Garate dialogaron j...,https://ideapais.cl/noticias-institucionales/i...
1,Candidatos a Gobernador Regional de la región ...,María José Aravena,"Oct 15, 2024","María José Aravena, Noticias Destacadas Inicio...",El debate se enfocó en temas de seguridad y co...,https://ideapais.cl/noticias-institucionales/c...
2,IdeaPaís invita a postular al II Congreso de P...,IdeaPais,"Jun 7, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",La instancia se realizará el 27 de septiembre ...,https://ideapais.cl/noticias-institucionales/i...
3,IdeaPaís inaugura sede regional en Los Lagos: ...,IdeaPais,"May 30, 2024","IdeaPais, Noticias IdeaPaís Los Lagos, Noticia...","En el evento, su director regional, Matías Dom...",https://ideapais.cl/noticias-institucionales/i...
4,IdeaPaís realiza debates entre candidatos a al...,María José Aravena,"Oct 17, 2024","María José Aravena, Noticias Destacadas Inicio...",El centro de estudios de inspiración social cr...,https://ideapais.cl/sin-categoria/ideapais-rea...
5,Con presencia de Evelyn Matthei el centro de e...,IdeaPais,"Sep 30, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",En el ex Congreso Nacional expusieron ex autor...,https://ideapais.cl/noticias-institucionales/c...
6,La versión 2024 del Encuentro de Líderes Jóven...,IdeaPais,"Jun 6, 2024","IdeaPais, Noticias Institucionales, Noticias O...","IdeaPaís, centro de estudios de inspiración so...",https://ideapais.cl/noticias-ohiggins/la-versi...
7,IdeaPaís y centros de estudios presentaron la ...,IdeaPais,"May 27, 2024","IdeaPais, Noticias Institucionales, 0 Comentarios",Con la presencia de la alcaldesa de Providenci...,https://ideapais.cl/noticias-institucionales/i...
8,Ex autoridades del Biobío entregan sus reflexi...,María José Aravena,"Oct 16, 2024","María José Aravena, Noticias Destacadas Inicio...",El panel de conversación se enfocó en los prin...,https://ideapais.cl/noticias-institucionales/e...
9,«En el Foco» de IdeaPaís analizó las eleccione...,IdeaPais,"Jul 17, 2024","IdeaPais, Noticias Destacadas Inicio, Noticias...",IdeaPaís llevó a cabo el evento online «Europa...,https://ideapais.cl/noticias-institucionales/e...


Crear otra funcion

In [16]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import pandas as pd

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def extract_info_from_url(driver, url):
    """
    Extrae información de un artículo dado su URL.
    
    Args:
        driver (WebDriver): El controlador de Selenium para navegar por la página.
        url (str): La URL del artículo del cual extraer la información.
    
    Returns:
        dict: Un diccionario con la información del artículo extraído.
    """
    driver.get(url)
    
    # Esperar a que cargue el contenido (se puede ajustar el tiempo de espera si es necesario)
    driver.implicitly_wait(3)  # Esperar hasta 3 segundos para que se cargue la página
    
    # Extraer la información de interés:
    try:
        # Extraer el título
        title = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_inner h1").text.strip()
    except:
        title = "No disponible"
    
    try:
        # Extraer el año y número de forma específica (de la clase correcta)
        year_issue = driver.find_element(By.CSS_SELECTOR, ".et_pb_text_0 .et_pb_text_inner p").text.strip()
    except:
        year_issue = "No disponible"
    
    try:
        # Extraer todo el contenido del artículo, evitando que incluya el "Año y Número"
        # Asegurarse de que el selector no esté extrayendo elementos no deseados.
        paragraphs = driver.find_elements(By.CSS_SELECTOR, ".et_pb_text_2 .et_pb_text_inner p")
        content = "\n".join([p.text.strip() for p in paragraphs if p.text.strip() != ""])  # Unir todos los párrafos
    except:
        content = "No disponible"
    
    try:
        # Esperar hasta que el enlace de descarga esté presente
        download_link_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//a[text()='Descarga la revista aquí']"))
        )
        download_link = download_link_element.get_attribute("href")
    except Exception as e:
        print(f"Error al obtener el enlace de descarga: {e}")
        download_link = "No disponible"
    
    # Crear el diccionario con los datos
    article_data = {
        "Título": title,
        "Año y Número": year_issue,
        "Contenido": content,
        "Enlace de Descarga": download_link,
        "Enlace": url,
    }
    
    return article_data

def process_all_links(jg_df, headless=True):
    """
    Procesa todos los enlaces de un DataFrame 'jg_df', extrayendo la información de cada artículo
    y almacenándola en un nuevo DataFrame 'publicaciones_jg'.
    
    Args:
        jg_df (pd.DataFrame): DataFrame que contiene una columna 'Enlace' con los enlaces de los artículos.
        headless (bool): Determina si el modo headless debe estar activado o no. Por defecto es True.
    
    Returns:
        pd.DataFrame: DataFrame con la información extraída de todos los enlaces.
    """
    # Configurar las opciones de Chrome para el modo headless
    chrome_options = Options()
    
    if headless:
        chrome_options.add_argument("--headless")  # Activar modo headless (sin interfaz gráfica)
    
    chrome_options.add_argument("--disable-gpu")  # Deshabilitar la aceleración por hardware (útil en headless)
    chrome_options.add_argument("--no-sandbox")  # Puede ser necesario en ciertos entornos (como contenedores)

    # Abrir el webdriver con las opciones configuradas
    driver = webdriver.Chrome(options=chrome_options)

    # Crear una lista para almacenar los datos de los artículos
    publicaciones_jg = []

    try:
        for _, row in jg_df.iterrows():
            url = row['Enlace']
            print(f"Extrayendo datos de: {url}")
            
            # Extraer la información del artículo
            article_data = extract_info_from_url(driver, url)
            
            if article_data:
                publicaciones_jg.append(article_data)
        
        # Crear un DataFrame con los artículos extraídos
        publicaciones_jg_df = pd.DataFrame(publicaciones_jg)
        
        return publicaciones_jg_df
    
    except Exception as e:
        print(f"Error en el proceso: {str(e)}")
        return pd.DataFrame()
    
    finally:
        driver.quit()  # Cerrar el webdriver al finalizar

In [17]:
ip_lyr_contenido = process_all_links(ip_lyr, headless=True)
ip_lyr_contenido

Extrayendo datos de: https://ideapais.cl/revista-raices-la-familia-un-bien-publico-vi-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-dilemas-de-la-juventud-v-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-pensar-una-centro-derecha-reformista-iv-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-reformismo-y-constitucion-iii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-desafios-constitucionales-ii-edicion/
Extrayendo datos de: https://ideapais.cl/revista-raices-solidaridad-i-edicion/
Extrayendo datos de: https://ideapais.cl/producto/gonzalo-vial-politica-y-crisis-social-2da-edicion-ampliada/
Error al obtener el enlace de descarga: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF6A7556CB5+28821]
	(No symbol) [0x00007FF6A74C3840]
	(No symbol) [0x00007FF6A736578A]
	(No symbol) [0x00007FF6A73B91BE]
	(No symbol) [0x00007FF6A73B94AC]
	(No symbol) [0x00007FF6A7402647]
	(No symbol) [0x00007FF6A73DF33F]
	(No symbol) [0x0000

,Título,Año y Número,Contenido,Enlace de Descarga,Enlace
0,Revista Raíces «La familia: un bien público»,"AÑO 2024, Nº6",El motor del trabajo de IdeaPaís durante estos...,https://ideapais.cl/wp-content/uploads/2024/04...,https://ideapais.cl/revista-raices-la-familia-...
1,Revista Raíces «Dilemas de la juventud»,"AÑO 2023, Nº5",La quinta edición de Raíces está dedicada a re...,https://ideapais.cl/wp-content/uploads/2023/08...,https://ideapais.cl/revista-raices-dilemas-de-...
2,Revista Raíces «Pensar una centro derecha refo...,"AÑO 2022, Nº4","Este número de la revista Raíces, busca dar un...",https://ideapais.cl/wp-content/uploads/2023/02...,https://ideapais.cl/revista-raices-pensar-una-...
3,Revista Raíces «Reformismo y Constitución»,"AÑO 2022, Nº3",La tercera edición de la revista Raíces nos en...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-reformismo-...
4,Revista Raíces «Desafíos constitucionales»,"AÑO 2021, Nº2",La experiencia de los últimos años nos ha ense...,https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-desafios-co...
5,Revista Raíces «Solidaridad»,"AÑO 2020, Nº1","A diez años del nacimiento de IdeaPaís, presen...",https://ideapais.cl/wp-content/uploads/2022/08...,https://ideapais.cl/revista-raices-solidaridad...
6,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/gonzalo-vial-poli...
7,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/cuestiones-obreras/
8,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/constitucion-soli...
9,No disponible,No disponible,,No disponible,https://ideapais.cl/producto/por-que-ha-fracas...


## Consolidar archivos

In [18]:
import pandas as pd

# Lista de DataFrames
dataframes = [ip_pp, ip_ml, ip_dc, ip_elm_contenido, ip_lyr_contenido, ip_notins_contenido, ip_inic]

# Concatenar los DataFrames
df_concatenado = pd.concat(dataframes, ignore_index=True)

# Ver el DataFrame concatenado
df_concatenado

,Título,Enlace,Fecha Publicación,URL Imagen,Contenido,Fecha Consulta,Autor,Fecha de Publicación,Categorías,Año y Número,Enlace de Descarga
0,Reporte II Congreso de Políticas Públicas Prof...,https://ideapais.cl/wp-content/uploads/2024/10...,"1 de Oct, 2024",https://ideapais.cl/wp-content/uploads/2024/10...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
1,Presentación: Labores de cuidados en la encues...,https://ideapais.cl/wp-content/uploads/2024/09...,"3 de Sep, 2024",https://ideapais.cl/wp-content/uploads/2024/09...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
2,Chile desde las comunas: Propuestas locales pa...,https://ideapais.cl/wp-content/uploads/2024/08...,"28 de Ago, 2024",https://ideapais.cl/wp-content/uploads/2024/08...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
3,Radiografía: Zoom a la clase media chilena,https://ideapais.cl/wp-content/uploads/2024/07...,"18 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
4,Radiografía: Matrícula extranjera en el sistem...,https://ideapais.cl/wp-content/uploads/2024/07...,"5 de Jul, 2024",https://ideapais.cl/wp-content/uploads/2024/07...,,2024-11-29 01:29:43,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
97,Memoria 2020,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
98,Memoria 2022,https://indd.adobe.com/view/2cb08e30-369b-439e...,"Feb 16, 2024","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
99,Memoria 2019,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN
100,Memoria 2021,https://ideapais.cl/wp-content/uploads/2022/08...,"Ago 12, 2022","data:image/gif;base64,R0lGODlhAQABAAAAACH5BAEK...",,2024-11-29 01:31:33,NaN,NaN,NaN,NaN,NaN


In [19]:
df_final = df_concatenado.copy()

In [21]:
#df_final["Medio de comunicación"] = ""
df_final["Think tank"] = "Idea País"

df_final  = df_final[['Título', 'Enlace', 'Fecha Publicación', 'URL Imagen', 'Contenido', 'Autor', 'Fecha de Publicación', 'Categorías',
       'Año y Número', 'Enlace de Descarga']]

## Guardar archivos

In [22]:
df_final.to_excel("Idea País.xlsx", index = False)